### Crear dataframe

In [1]:
!pip -q install openpyxl

In [36]:
import pandas as pd
import hashlib

def calculate_total_points(row):
    """Calcula el total de puntos para una fila del DataFrame."""
    score_columns = ['Puntuación 1', 'Puntuación 2', 'Puntuación 3']
    return sum(row[col] for col in score_columns if pd.notnull(row[col]))

def calculate_total_points_to_consider(row):
    """Calcula el total de puntos a considerar para una fila del DataFrame."""
    point_columns = ['Punto 1', 'Punto 2', 'Punto 3']
    return sum(1 for col in point_columns if row[col] != '')


# Cargar el archivo CSV
df = pd.read_csv('/workspaces/codespaces-jupyter/data/Pauta-Control4.csv')

df = df.fillna('')

df = df.drop(df.columns[2], axis=1)

df.columns = ['Tópico', 'Pregunta', 'Punto 1', 'Puntuación 1', 'Punto 2', 'Puntuación 2', 'Punto 3', 'Puntuación 3']

# Generar un ID único para cada pregunta usando una función de hash
df['ID'] = df['Pregunta'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

# Convertir las columnas de puntuación a numéricas para poder sumarlas
df['Puntuación 1'] = pd.to_numeric(df['Puntuación 1'], errors='coerce').fillna(0)
df['Puntuación 2'] = pd.to_numeric(df['Puntuación 2'], errors='coerce').fillna(0)
df['Puntuación 3'] = pd.to_numeric(df['Puntuación 3'], errors='coerce').fillna(0)

# Calcular el total de puntos para cada pregunta y el total de puntos a considerar
df['Total Puntos'] = df.apply(calculate_total_points, axis=1)
df['Total Puntos a Considerar'] = df.apply(calculate_total_points_to_consider, axis=1)

# Reordenar las columnas
df = df[['Tópico', 'Pregunta', 'Total Puntos', 'Total Puntos a Considerar', 'Punto 1', 'Puntuación 1', 'Punto 2', 'Puntuación 2', 'Punto 3', 'Puntuación 3', 'ID']]

# Guardar el DataFrame como CSV y Excel en carpeta data
df.to_csv('/workspaces/codespaces-jupyter/data/pauta.csv', index=False)
df.to_excel('/workspaces/codespaces-jupyter/data/pauta.xlsx', index=False)


### Generar preguntas a partir de la pauta

In [37]:
!pip -q install langchain openai tiktoken cohere

In [57]:
def create_scoring_guideline(row):
    # Inserta la pauta de corrección en el formato de la plantilla
    points = [f"Punto {i+1}: {row[f'Punto {i+1}']} (Puntuación: {row[f'Puntuación {i+1}']})"
              for i in range(3) if pd.notnull(row[f'Punto {i+1}'])]
    return ', '.join(points)

In [39]:
df = pd.read_csv('/workspaces/codespaces-jupyter/data/pauta.csv')

df['Scoring Guideline'] = df.apply(create_scoring_guideline, axis=1)

# Guardar el DataFrame como CSV y Excel en carpeta data
df.to_csv('/workspaces/codespaces-jupyter/data/pauta.csv', index=False)
df.to_excel('/workspaces/codespaces-jupyter/data/pauta.xlsx', index=False)


#### Configuracion del prompt

In [46]:
from pydantic import BaseModel, Field, validator
from typing import List
import re
import json

class QuestionVariant(BaseModel):
    ID: str = Field(description="Unique identifier for the question")
    Pregunta_original: str = Field(description="Original question text")
    Variante: str = Field(description="Generated question variant")
    
    # Validar que la pregunta termine con un signo de interrogación
    @validator('Variante')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field


def parse_gpt3_response(generated_text):
    #Parsea la respuesta de GPT-3 y devuelve una lista de variantes de preguntas.

    parsed_response = json.loads(generated_text)

    # Extraer las variantes de la respuesta
    variants = [value for key, value in parsed_response.items() if "Variante" in key]

    return variants




# Define a function to create a prompt
def create_prompt(row):
    #Crea el prompt para una fila del DataFrame.
    
    return f"""
    Genera una forma de plantear la siguiente pregunta , que está relacionada con el tema '{row['Tópico']}' :

    "{row['Pregunta']}"

    Asegúrate de que las variantes siguen estas pautas de puntuación, ya que seran evaluadas con la exactamente esta pauta:

    "{row['Scoring Guideline']}"

    El formato de respuesta debe ser JSON.
    Se espera este formato:

    ```JSON
    {{
        "ID": "sample_id_variant",
        "Pregunta_original": "En un monticulo de maximos, donde se encuentra el valor maximo?",
        "Variante": "En el contexto de una empresa de comercio electrónico que utiliza un montículo de máximos para manejar las puntuaciones de los productos en su plataforma, ¿en qué posición se podría encontrar el producto con la puntuación más alta?"
    }}
    ```
    """

sample_row = {
    'Tópico': 'Montículo',
    'Pregunta': 'En un montículo de máximos, ¿dónde se encuentra el valor máximo?',
    'Scoring Guideline': 'End with a question mark.',
    'ID': 'sample_id'
}

# Crear el prompt para la fila de ejemplo
prompt = create_prompt(sample_row)
print(f"Generated Prompt:\n{prompt}")


# Simular la respuesta de GPT-3
simulate_gpt3_response = """
{
    "ID": "sample_id_variant",
    "Pregunta_original": "En un monticulo de maximos, donde se encuentra el valor maximo?",
    "Variante": "En el contexto de una empresa de comercio electrónico que utiliza un montículo de máximos para manejar las puntuaciones de los productos en su plataforma, ¿en qué posición se podría encontrar el producto con la puntuación más alta?"
}
"""

# Parsear la respuesta simulada
parsed_response = parse_gpt3_response(simulate_gpt3_response)

# Crear una lista vacía para almacenar las variantes de preguntas
question_variants = []

# Crear una instancia de QuestionVariant para cada variante de pregunta
for variant in parsed_response:
    question_variant = QuestionVariant(ID=sample_row['ID'], Pregunta_original=sample_row['Pregunta'], Variante=variant)
    question_variants.append(question_variant)

# Imprimir las variantes de preguntas
for question_variant in question_variants:
    print(question_variant.json(indent=2))

Generated Prompt:

    Genera una forma de plantear la siguiente pregunta , que está relacionada con el tema 'Montículo' :

    "En un montículo de máximos, ¿dónde se encuentra el valor máximo?"

    Asegúrate de que las variantes siguen estas pautas de puntuación, ya que seran evaluadas con la exactamente esta pauta:

    "End with a question mark."

    El formato de respuesta debe ser JSON.
    Se espera este formato:

    ```JSON
    {
        "ID": "sample_id_variant",
        "Pregunta_original": "En un monticulo de maximos, donde se encuentra el valor maximo?",
        "Variante": "En el contexto de una empresa de comercio electrónico que utiliza un montículo de máximos para manejar las puntuaciones de los productos en su plataforma, ¿en qué posición se podría encontrar el producto con la puntuación más alta?"
    }
    ```
    
{
  "ID": "sample_id",
  "Pregunta_original": "En un mont\u00edculo de m\u00e1ximos, \u00bfd\u00f3nde se encuentra el valor m\u00e1ximo?",
  "Variante":

#### Funcion principal

In [47]:
import openai

def generate_question_variants(df, num_iterations=1):
    # Crear dataframe donde seran guardadas las variantes
    variants_df = pd.DataFrame(columns=["ID", "Pregunta original", "Variante"])
    
    # Abrir archivo donde seran guardadas las variantes por si se interrumpe el proceso
    with open("gpt3_responses.txt", "a") as f:
        # Iterar sobre cada fila del dataframe
        for _, row in df.iterrows():
            # Crear el prompt
            prompt = create_prompt(row)
            
            # Genera y guarda las variantes siguiendo el formato JSON
            for _ in range(num_iterations):
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=450
                )
                generated_text = response['choices'][0]['message']['content'].strip()
                variants = parse_gpt3_response(generated_text)
                
                # Anadir las variantes al dataframe
                for variant in variants:
                    variant_df = pd.DataFrame([{"ID": row["ID"], "Pregunta original": row["Pregunta"], "Variante": variant}], columns=["ID", "Pregunta original", "Variante"])
                    variants_df = pd.concat([variants_df, variant_df], ignore_index=True)
                

                # Imprimir el prompt y el texto generado
                print("Prompt used:")
                print(prompt)
                print()
                
                print("Generated Text:")
                print(generated_text)
                print()
                
                # Guardar el prompt y el texto generado en un archivo
                f.write(f"ID: {row['ID']}\n")
                f.write(f"Prompt: {prompt}\n")
                f.write(f"Generated Text: {generated_text}\n\n")
    
    # Devolver el dataframe con las variantes
    return variants_df


### Generar preguntas variantes

In [48]:
# Genera las preguntas variantes a partir de la pauta
variants_df = generate_question_variants(df, num_iterations=1)

Prompt used:

    Genera una forma de plantear la siguiente pregunta , que está relacionada con el tema 'Montículo' :

    "En un montículo de máximos, ¿dónde se encuentra el máximo y el mínimo elemento?"

    Asegúrate de que las variantes siguen estas pautas de puntuación, ya que seran evaluadas con la exactamente esta pauta:

    "Punto 1: El máximo se encuentra en la raíz (Puntuación: 1.0), Punto 2: El mínimo se encuentra en una hoja (que no es equivalente a  dentro del arreglo) (Puntuación: 1.0)"

    El formato de respuesta debe ser JSON.
    Se espera este formato:

    ```JSON
    {
        "ID": "sample_id_variant",
        "Pregunta_original": "En un monticulo de maximos, donde se encuentra el valor maximo?",
        "Variante": "En el contexto de una empresa de comercio electrónico que utiliza un montículo de máximos para manejar las puntuaciones de los productos en su plataforma, ¿en qué posición se podría encontrar el producto con la puntuación más alta?"
    }
    ```
   

In [49]:
# Guarda el dataframe como CSV
variants_df.to_csv('variants.csv', index=False)
variants_df


,ID,Pregunta original,Variante
0,c4f6c99ba30f4d5f46848e0e601c89658a56f8aa7ec746...,"En un montículo de máximos, ¿dónde se encuentr...",Si se tiene un montículo de máximos que repres...
1,ef837014740ada5351f39db6910208979dac8b81cd79a1...,"En un montículo de máximos, ¿cómo encuentro el...","En un montículo de máximos, ¿cómo puedo verifi..."
2,f057dab5ea2d18b3914d62f6fd8025b37b62585756582a...,¿Qué propiedad básica de la estructura de árbo...,En el contexto de una tienda en línea que util...
3,cd6b2697a4a7ba4a519314646d2afba3f506f2ffcfb745...,¿Cómo se representa un montículo binario en me...,En el contexto de un programa de gestión de da...
4,8df7b012e5f89cf02629e4f9193e95768b083d18b5c983...,¿Por qué se dice que la inserción en un montíc...,¿Cuál es la justificación de que la inserción ...
...,...,...,...
63,95c338c15b9b66a09e44cbb4e73a915cf5ac3fe519bcb2...,¿Qué TDA(s) usaría para construir un sistema d...,"En el contexto de un supermercado, ¿qué TDA(s)..."
64,7c3edcc9d0a4500d83781b722683dd3bb450cb5f3e763b...,¿Qué TDA(s) usaría para realizar operaciones d...,En el contexto de una biblioteca digital que n...
65,fb1016134fc0a8a9523de70f97f12662ffd2d0c3673397...,¿Qué TDA(s) usaría para programar la ruta más ...,En el desarrollo de una aplicación de navegaci...
66,7f8ee62de6d70a52e7bed692bf7575b88151e9cc2847ef...,¿Qué TDA(s) usaría para mantener el seguimient...,En el contexto de un software de gestión de pr...


#### Funciones para encontrar pauta a pregunta generada

In [53]:
def get_index_by_id(df, id_value):
    index = df[df['ID'] == id_value].index
    
    return index[0] if len(index) > 0 else None

index = get_index_by_id(df, variants_df.iloc[0]['ID'])
print(index)


0


En este caso el id es el mismo, ya que al usar el codigo el numero de iteracion es solo uno.

Se sugiere aumentar el numero de iteraciones si se requieren de mas variantes, otra sugerencia a realizar, es pedirle en el prompt al modelo mas de una variante. en las primeras versiones se pedian mas de una pregunta en la misma iteracion pero esto fue modificado para que sea mas facil crear el prototipo.

In [ ]:
def find_matching_rows(df, variants_df):
    merged_df = pd.merge(df, variants_df, on="ID", how="inner")
    matching_indices = merged_df.index

    return matching_indices


---